In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Données fournies
data = [
    ('Abidjan', 'Cocody', 'Riviera', 5, 4, 3, 250, 130000000, 'Grande villa avec piscine et garage', 'Villa', 'En vente', '5.3581, -3.9983', 1, 40, 220),
    ('San-Pédro', 'Bardot', 'Zone Portuaire', 3, 2, 1, 150, 85000000, 'Maison moderne près du port', 'Maison', 'Vendu', '4.7511, -6.6364', 2, 25, 150),
    ('Bouaké', 'Nimbo', 'Quartier Résidentiel', 4, 3, 2, 180, 90000000, 'Maison avec jardin et garage', 'Maison', 'En vente', '7.6903, -5.0327', 3, 30, 180),
    ('Yamoussoukro', 'Morofé', 'Quartier des Ambassades', 5, 4, 3, 300, 140000000, 'Villa de prestige avec piscine', 'Villa', 'Vendu', '6.8218, -5.2769', 4, 50, 240),
    ('Daloa', 'Brébo', 'Quartier Administratif', 3, 2, 1, 130, 80000000, 'Appartement spacieux avec balcon', 'Appartement', 'En vente', '6.8792, -6.4531', 5, 20, 130),
    ('Man', 'Abattoir', 'Quartier Militaire', 4, 3, 2, 200, 95000000, 'Maison familiale avec grande cour', 'Maison', 'Vendu', '7.4165, -7.5563', 6, 35, 160),
    ('Gagnoa', 'Gagnoa', 'Quartier Commercial', 2, 1, 1, 100, 70000000, 'Petit appartement moderne', 'Appartement', 'En vente', '6.1328, -5.9512', 7, 15, 90),
    ('Korhogo', 'Péléforo', 'Quartier Central', 3, 2, 2, 160, 85000000, 'Maison avec terrasse et jardin', 'Maison', 'Vendu', '9.4583, -5.6295', 1, 28, 150),
    ('Abengourou', 'Indénié', 'Quartier Administratif', 3, 2, 1, 140, 75000000, 'Appartement rénové', 'Appartement', 'En vente', '6.7320, -3.4918', 2, 18, 100),
    ('Aboisso', 'Bia Sud', 'Quartier Administratif', 5, 4, 3, 240, 120000000, 'Grande villa avec piscine', 'Villa', 'Vendu', '5.4730, -3.2065', 3, 38, 200),
    ('Divo', 'Gogobro', 'Quartier Résidentiel', 2, 1, 1, 90, 60000000, 'Maison près du marché', 'Maison', 'En vente', '5.8380, -5.3582', 4, 13, 80),
    ('Abidjan', 'Plateau', 'Centre-ville', 4, 3, 2, 180, 100000000, 'Appartement de luxe avec vue', 'Appartement', 'Vendu', '5.3043, -4.0027', 5, 30, 190)
]

# Conversion en DataFrame
columns = ['Ville', 'Commune', 'Quartier', 'Nombre_de_pieces', 'Nombre_de_chambres', 'Nombre_de_salles_de_bain', 'metre_carre', 'Prix_unitaire', 'Descriptions', 'Type_de_maison', 'Statut_maison', 'GPS', 'Idproprietaire', 'Likes', 'Nombre_de_vue']
df = pd.DataFrame(data, columns=columns)

# Prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Nombre_de_pieces', 'Nombre_de_chambres', 'metre_carre', 'Prix_unitaire', 'Likes', 'Nombre_de_vue']),
        ('cat', OneHotEncoder(), ['Ville', 'Commune', 'Quartier', 'Type_de_maison', 'Statut_maison'])
    ]
)

# Pipeline de prétraitement
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Transformation des données
processed_data = pipeline.fit_transform(df)

# Extraction des noms de colonnes après transformation
transformed_feature_names = list(pipeline.named_steps['preprocessor'].transformers_[0][2]) + \
                            list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(['Ville', 'Commune', 'Quartier', 'Type_de_maison', 'Statut_maison']))

print("Noms de colonnes :", transformed_feature_names)
print("Données transformées :\n", processed_data)

# Conversion des données transformées en DataFrame avec les noms de colonnes appropriés
processed_df = pd.DataFrame(processed_data,)


from sklearn.metrics.pairwise import cosine_similarity

# Calcul des similarités cosinus entre les maisons basées sur leurs caractéristiques
content_similarity = cosine_similarity(processed_data, processed_data)


# Création de la matrice d'interactions utilisateur-bien (likes + vues)
interaction_matrix = df[['Likes', 'Nombre_de_vue']].values

# Calcul des similarités cosinus entre les utilisateurs
collaborative_similarity = cosine_similarity(interaction_matrix, interaction_matrix)


# Poids des modèles
alpha = 0.5  # Poids pour le modèle de contenu
beta = 0.5   # Poids pour le modèle collaboratif

# Combinaison des similarités
hybrid_similarity = alpha * content_similarity + beta * collaborative_similarity

# Sélection d'un utilisateur (par exemple, utilisateur 1)
user_index = 0

# Scores de recommandation pour tous les biens
user_hybrid_scores = hybrid_similarity[user_index]

# Trier les biens par score de recommandation décroissant
recommended_items_indices = np.argsort(user_hybrid_scores)[::-1]

# Afficher les biens recommandés avec leurs scores
print("Biens recommandés pour l'utilisateur 1:")
for idx in recommended_items_indices:
    print(f"Description : {df.iloc[idx]['Descriptions']}")
    print(f"Ville : {df.iloc[idx]['Ville']}, Commune : {df.iloc[idx]['Commune']}, Quartier : {df.iloc[idx]['Quartier']}")
    print(f"Nombre de pièces : {df.iloc[idx]['Nombre_de_pieces']}, Nombre de chambres : {df.iloc[idx]['Nombre_de_chambres']}")
    print(f"Prix unitaire : {df.iloc[idx]['Prix_unitaire']}")
    print(f"Score de recommandation : {user_hybrid_scores[idx]}")
    print("-------------")



In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Données fournies
data = [
    ('Abidjan', 'Cocody', 'Riviera', 5, 4, 3, 250, 130000000, 'Grande villa avec piscine et garage', 'Villa', 'En vente', '5.3581, -3.9983', 1, 40, 220),
    ('San-Pédro', 'Bardot', 'Zone Portuaire', 3, 2, 1, 150, 85000000, 'Maison moderne près du port', 'Maison', 'Vendu', '4.7511, -6.6364', 2, 25, 150),
    ('Bouaké', 'Nimbo', 'Quartier Résidentiel', 4, 3, 2, 180, 90000000, 'Maison avec jardin et garage', 'Maison', 'En vente', '7.6903, -5.0327', 3, 30, 180),
    ('Yamoussoukro', 'Morofé', 'Quartier des Ambassades', 5, 4, 3, 300, 140000000, 'Villa de prestige avec piscine', 'Villa', 'Vendu', '6.8218, -5.2769', 4, 50, 240),
    ('Daloa', 'Brébo', 'Quartier Administratif', 3, 2, 1, 130, 80000000, 'Appartement spacieux avec balcon', 'Appartement', 'En vente', '6.8792, -6.4531', 5, 20, 130),
    ('Man', 'Abattoir', 'Quartier Militaire', 4, 3, 2, 200, 95000000, 'Maison familiale avec grande cour', 'Maison', 'Vendu', '7.4165, -7.5563', 6, 35, 160),
    ('Gagnoa', 'Gagnoa', 'Quartier Commercial', 2, 1, 1, 100, 70000000, 'Petit appartement moderne', 'Appartement', 'En vente', '6.1328, -5.9512', 7, 15, 90),
    ('Korhogo', 'Péléforo', 'Quartier Central', 3, 2, 2, 160, 85000000, 'Maison avec terrasse et jardin', 'Maison', 'Vendu', '9.4583, -5.6295', 1, 28, 150),
    ('Abengourou', 'Indénié', 'Quartier Administratif', 3, 2, 1, 140, 75000000, 'Appartement rénové', 'Appartement', 'En vente', '6.7320, -3.4918', 2, 18, 100),
    ('Aboisso', 'Bia Sud', 'Quartier Administratif', 5, 4, 3, 240, 120000000, 'Grande villa avec piscine', 'Villa', 'Vendu', '5.4730, -3.2065', 3, 38, 200),
    ('Divo', 'Gogobro', 'Quartier Résidentiel', 2, 1, 1, 90, 60000000, 'Maison près du marché', 'Maison', 'En vente', '5.8380, -5.3582', 4, 13, 80),
    ('Abidjan', 'Plateau', 'Centre-ville', 4, 3, 2, 180, 100000000, 'Appartement de luxe avec vue', 'Appartement', 'Vendu', '5.3043, -4.0027', 5, 30, 190)
]

# Conversion en DataFrame
columns = ['Ville', 'Commune', 'Quartier', 'Nombre_de_pieces', 'Nombre_de_chambres', 'Nombre_de_salles_de_bain', 'metre_carre', 'Prix_unitaire', 'Descriptions', 'Type_de_maison', 'Statut_maison', 'GPS', 'IdUtilisateur', 'Likes', 'Nombre_de_vue']
df = pd.DataFrame(data, columns=columns)

# Prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Nombre_de_pieces', 'Nombre_de_chambres', 'metre_carre', 'Prix_unitaire', 'Likes', 'Nombre_de_vue']),
        ('cat', OneHotEncoder(), ['Ville', 'Commune', 'Quartier', 'Type_de_maison', 'Statut_maison'])
    ]
)

# Pipeline de prétraitement
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Transformation des données
processed_data = pipeline.fit_transform(df)

# Extraction des noms de colonnes après transformation
transformed_feature_names = list(pipeline.named_steps['preprocessor'].transformers_[0][2]) + \
                            list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(['Ville', 'Commune', 'Quartier', 'Type_de_maison', 'Statut_maison']))

print("Noms de colonnes :", transformed_feature_names)
print("Données transformées :\n", processed_data)

# Conversion des données transformées en DataFrame avec les noms de colonnes appropriés
processed_df = pd.DataFrame(processed_data,)


Noms de colonnes : ['Nombre_de_pieces', 'Nombre_de_chambres', 'metre_carre', 'Prix_unitaire', 'Likes', 'Nombre_de_vue', 'Ville_Abengourou', 'Ville_Abidjan', 'Ville_Aboisso', 'Ville_Bouaké', 'Ville_Daloa', 'Ville_Divo', 'Ville_Gagnoa', 'Ville_Korhogo', 'Ville_Man', 'Ville_San-Pédro', 'Ville_Yamoussoukro', 'Commune_Abattoir', 'Commune_Bardot', 'Commune_Bia Sud', 'Commune_Brébo', 'Commune_Cocody', 'Commune_Gagnoa', 'Commune_Gogobro', 'Commune_Indénié', 'Commune_Morofé', 'Commune_Nimbo', 'Commune_Plateau', 'Commune_Péléforo', 'Quartier_Centre-ville', 'Quartier_Quartier Administratif', 'Quartier_Quartier Central', 'Quartier_Quartier Commercial', 'Quartier_Quartier Militaire', 'Quartier_Quartier Résidentiel', 'Quartier_Quartier des Ambassades', 'Quartier_Riviera', 'Quartier_Zone Portuaire', 'Type_de_maison_Appartement', 'Type_de_maison_Maison', 'Type_de_maison_Villa', 'Statut_maison_En vente', 'Statut_maison_Vendu']
Données transformées :
   (0, 0)	1.3654728591342478
  (0, 1)	1.3654728591342

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcul des similarités cosinus entre les maisons basées sur leurs caractéristiques
content_similarity = cosine_similarity(processed_data, processed_data)


In [3]:
# Création de la matrice d'interactions utilisateur-bien (likes + vues)
interaction_matrix = df[['Likes', 'Nombre_de_vue']].values

# Calcul des similarités cosinus entre les utilisateurs
collaborative_similarity = cosine_similarity(interaction_matrix, interaction_matrix)


In [4]:
# Poids des modèles
alpha = 0.5  # Poids pour le modèle de contenu
beta = 0.5   # Poids pour le modèle collaboratif

# Combinaison des similarités
hybrid_similarity = alpha * content_similarity + beta * collaborative_similarity


In [5]:
# Sélection d'un utilisateur (par exemple, utilisateur 1)
user_index = 0

# Scores de recommandation pour tous les biens
user_hybrid_scores = hybrid_similarity[user_index]

# Trier les biens par score de recommandation décroissant
recommended_items_indices = np.argsort(user_hybrid_scores)[::-1]

# Afficher les biens recommandés avec leurs scores
print("Biens recommandés pour l'utilisateur 1:")
for idx in recommended_items_indices:
    print(f"Description : {df.iloc[idx]['Descriptions']}")
    print(f"Ville : {df.iloc[idx]['Ville']}, Commune : {df.iloc[idx]['Commune']}, Quartier : {df.iloc[idx]['Quartier']}")
    print(f"Nombre de pièces : {df.iloc[idx]['Nombre_de_pieces']}, Nombre de chambres : {df.iloc[idx]['Nombre_de_chambres']}")
    print(f"Prix unitaire : {df.iloc[idx]['Prix_unitaire']}")
    print(f"Score de recommandation : {user_hybrid_scores[idx]}")
    print("-------------")


Biens recommandés pour l'utilisateur 1:
Description : Grande villa avec piscine et garage
Ville : Abidjan, Commune : Cocody, Quartier : Riviera
Nombre de pièces : 5, Nombre de chambres : 4
Prix unitaire : 130000000
Score de recommandation : 0.9999999999999999
-------------
Description : Villa de prestige avec piscine
Ville : Yamoussoukro, Commune : Morofé, Quartier : Quartier des Ambassades
Nombre de pièces : 5, Nombre de chambres : 4
Prix unitaire : 140000000
Score de recommandation : 0.8816576252553757
-------------
Description : Grande villa avec piscine
Ville : Aboisso, Commune : Bia Sud, Quartier : Quartier Administratif
Nombre de pièces : 5, Nombre de chambres : 4
Prix unitaire : 120000000
Score de recommandation : 0.8513864671161875
-------------
Description : Appartement de luxe avec vue
Ville : Abidjan, Commune : Plateau, Quartier : Centre-ville
Nombre de pièces : 4, Nombre de chambres : 3
Prix unitaire : 100000000
Score de recommandation : 0.6868207998801291
-------------
Des